In [1]:
# Update iterator algorithm; and making recommendations; 
import collections
import copy
import itertools
import libsbml
import numpy as np
import os
import pickle
import pandas as pd
import sys
import matplotlib.pyplot as plt
%matplotlib inline  

BIOMD_12 = 'BIOMD0000000012.xml'
BASE_DIR = '/Users/woosubshin/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
ALGO_DIR = os.path.join(DATA_DIR, "algo")
CHEBI_DIR = os.path.join(DATA_DIR, "chebi")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubshin/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")

PROJ_DIR = os.path.join(os.getcwd(), os.pardir)
AMAS_DIR = os.path.join(PROJ_DIR, "AMAS")
sys.path.append(PROJ_DIR)

SUPPL_DIR = os.path.join(PROJ_DIR, os.pardir, "AMAS_suppl")
ACCURACY_DIR = os.path.join(SUPPL_DIR, "data_for_credibility")

from AMAS import species_annotation as sa
from AMAS import reaction_annotation as ra
from AMAS import recommender
from AMAS import constants as cn
from AMAS import iterator as it
from AMAS import tools

# Should save & load predicted species & reaction annotations

In [12]:
# get recom 
one_fpath = os.path.join(BIOMODEL_DIR, 'BIOMD0000000190.xml')
reader = libsbml.SBMLReader()
document = reader.readSBML(one_fpath)
model = document.getModel()
recom = recommender.Recommender(libsbml_fpath=one_fpath)

SPECIES_SAM = 'SAM'
REACTION_SAMDC = 'SAMdc'

In [6]:
recom.getSpeciesIDs()

['SAM', 'A', 'P', 'S', 'D', 'aS', 'aD', 'Met', 'ORN', 'AcCoA', 'CoA']

In [29]:
s = recom.recommendSpecies(['A'])

                               A                                
+----+--------------+---------------+--------------------------+
|    | annotation   |   match score | label                    |
+====+==============+===============+==========================+
|  1 | CHEBI:15625  |         1.000 | S-adenosylmethioninamine |
+----+--------------+---------------+--------------------------+



In [30]:
RESULT_RECOM = cn.Recommendation('R_PFK',
                                 [('RHEA:12420', 0.6), ('RHEA:13377', 0.6)],
                                 ['https://www.rhea-db.org/rhea/12420', 'https://www.rhea-db.org/rhea/13377'],
                                 ['tagatose-6-phosphate kinase activity', 'phosphoglucokinase activity'])

In [31]:
res = recom.getMarkdownFromRecommendation(rec=RESULT_RECOM,
                                               show_url=False)

In [32]:
res

'                                   R_PFK                                    \n+----+--------------+---------------+--------------------------------------+\n|    | annotation   |   match score | label                                |\n+====+==============+===============+======================================+\n|  1 | RHEA:12420   |         0.600 | tagatose-6-phosphate kinase activity |\n+----+--------------+---------------+--------------------------------------+\n|  2 | RHEA:13377   |         0.600 | phosphoglucokinase activity          |\n+----+--------------+---------------+--------------------------------------+'

In [21]:
recom.getReactionIDs()

['ODC',
 'SAMdc',
 'SSAT_for_S',
 'SSAT_for_D',
 'PAO_for_aD',
 'PAO_for_aS',
 'SpdS',
 'SpmS',
 'MAT',
 'VCoA',
 'VacCoA',
 'P_efflux',
 'aD_efflux']

In [23]:
res2 = recom.getReactionRecommendation(pred_id='aD_efflux', get_df=True)

In [24]:
res2

,annotation,match score,label
aD_efflux,,,
1,RHEA:23928,0.25,acetylspermidine deacetylase activity
2,RHEA:27578,0.25,(R)-N-tert-butylpiperazine-2-carboxamide + H2O...
3,RHEA:51900,0.25,H2O + N(1)-acetylspermidine = acetate + spermi...


In [18]:
res2 = recom.getReactionRecommendation(pred_id='CoA', get_df=True)
df2 = recom.autoSelectAnnotation(res2, 0.6)
print(df2.shape[0], 0)

KeyError: 'CoA'

In [17]:
df2

,annotation,match score,label
SAMdc,,,
1,RHEA:28274,1.0,(S)-S-adenosyl-L-methionine = (R)-S-adenosyl-L...


In [ ]:
getReactionListRecommendation(pred_ids=reaction_list,
                                             mssc=mssc,
                                             cutoff=cutoff,
                                             get_df=True)

In [3]:
mssc = 'top'
cutoff = 0.0
model_mean = True

In [7]:
REACTION_ODC = 'ODC'
REACTION_SAMDC = 'SAMdc'

In [8]:
reacs = recom.getReactionListRecommendation(pred_ids=[REACTION_ODC, REACTION_SAMDC])

In [9]:
reacs

[Recommendation(id='ODC', candidates=[('RHEA:11584', 1.0), ('RHEA:14893', 1.0), ('RHEA:28827', 1.0), ('RHEA:71199', 1.0), ('RHEA:71527', 1.0), ('RHEA:72135', 1.0), ('RHEA:73471', 1.0)], urls=['https://www.rhea-db.org/rhea/11584', 'https://www.rhea-db.org/rhea/14893', 'https://www.rhea-db.org/rhea/28827', 'https://www.rhea-db.org/rhea/71199', 'https://www.rhea-db.org/rhea/71527', 'https://www.rhea-db.org/rhea/72135', 'https://www.rhea-db.org/rhea/73471'], labels=['ornithine racemase activity', 'D-ornithine 4,5-aminomutase activity', 'L-ornithine(out) + putrescine(in) = L-ornithine(in) + putrescine(out)', 'L-ornithine(in) = L-ornithine(out)', 'thymine(out) = thymine(in)', 'putrescine(out) = putrescine(in)', 'D-ornithine(in) + L-ornithine(out) = D-ornithine(out) + L-ornithine(in)']),
 Recommendation(id='SAMdc', candidates=[('RHEA:28274', 1.0)], urls=['https://www.rhea-db.org/rhea/28274'], labels=['(S)-S-adenosyl-L-methionine = (R)-S-adenosyl-L-methionine'])]

In [5]:
recom.getReactionListRecommendation(pred_ids=['ODC'], get_df=True)

[     annotation  match score  \
 ODC                            
 1    RHEA:11584          1.0   
 2    RHEA:14893          1.0   
 3    RHEA:28827          1.0   
 4    RHEA:71199          1.0   
 5    RHEA:71527          1.0   
 6    RHEA:72135          1.0   
 7    RHEA:73471          1.0   
 
                                                  label  
 ODC                                                     
 1                          ornithine racemase activity  
 2                 D-ornithine 4,5-aminomutase activity  
 3    L-ornithine(out) + putrescine(in) = L-ornithin...  
 4                   L-ornithine(in) = L-ornithine(out)  
 5                           thymine(out) = thymine(in)  
 6                     putrescine(out) = putrescine(in)  
 7    D-ornithine(in) + L-ornithine(out) = D-ornithi...  ]

In [6]:
recom.reactions.candidates

{'ODC': [('RHEA:11584', 1.0),
  ('RHEA:14893', 1.0),
  ('RHEA:28827', 1.0),
  ('RHEA:71199', 1.0),
  ('RHEA:71527', 1.0),
  ('RHEA:72135', 1.0),
  ('RHEA:73471', 1.0)]}

In [4]:
res = recom.getSpeciesRecommendation(pred_str='hydrogen',
                                     mssc='top',
                                     cutoff=0.0,
                                     get_df=True)
res

,annotation,match score,label
hydrogen,,,
1,CHEBI:18276,1.0,dihydrogen
2,CHEBI:29235,1.0,hydrogen(.)
3,CHEBI:49637,1.0,hydrogen atom


In [5]:
from AMAS.reaction_annotation import REF_MAT
import operator

reacs=None
ref_mat=REF_MAT
update=False

In [6]:
recom.reactions.reaction_components

{'ODC': ['P', 'ORN'],
 'SAMdc': ['A', 'SAM'],
 'SSAT_for_S': ['aS', 'CoA', 'S', 'AcCoA'],
 'SSAT_for_D': ['CoA', 'D', 'AcCoA', 'aD'],
 'PAO_for_aD': ['P', 'aD'],
 'PAO_for_aS': ['aS', 'D'],
 'SpdS': ['A', 'D', 'P'],
 'SpmS': ['A', 'S', 'D'],
 'MAT': ['Met', 'SAM'],
 'VCoA': ['CoA', 'AcCoA'],
 'VacCoA': ['CoA', 'AcCoA'],
 'P_efflux': ['P'],
 'aD_efflux': ['aD']}

In [7]:
recom2 = recommender.Recommender(libsbml_fpath=ecoli_fpath)

In [9]:
recom2.reactions.reaction_components['R_PFK']

['M_f6p_c', 'M_fdp_c', 'M_atp_c', 'M_h_c', 'M_adp_c']

In [10]:
spec_ids = recom2.reactions.reaction_components['R_PFK']
spec_res = recom2.getSpeciesListRecommendation(pred_ids=spec_ids)
recom2.species.formula

{'M_f6p_c': ['C6O9P'],
 'M_fdp_c': ['C6O12P2'],
 'M_atp_c': ['C30N4O29P3'],
 'M_h_c': ['H', 'C6N3O2', 'C6N3O', '[3He]', 'C12N6O3'],
 'M_adp_c': ['C30O8P', 'C39O8P']}

In [16]:
reac_res = recom2.reactions.getRScores(spec_dict=recom2.species.formula,
                           reacs=['R_PFK'])['R_PFK']
res_vals = [val[1] for val in reac_res]

In [17]:
np.max(res_vals)

1.0

In [18]:
np.min(res_vals)

0.0

In [ ]:
# First, collect all species IDs to annotate
specs_to_annotate = list(set(itertools.chain(*[self.reactions.reaction_components[val] \
                                               for val in pred_ids])))
if use_exist_species_annotation:
  pred_formulas = {val:self.species.exist_annotation_formula[val] \
                   for val in specs_to_annotate \
                   if val in self.species.exist_annotation_formula.keys()}
else:
  pred_formulas = {}
remaining_species = [val for val in specs_to_annotate if val not in pred_formulas.keys()]
# Get annotation of collected species
if len(remaining_species) > 0:
  if spec_res:
    spec_results = [val for val in spec_res if val.id in remaining_species]
  else:
    # not updating as it is a temporary prediction (and not for user)
    spec_results = self.getSpeciesListRecommendation(pred_ids=remaining_species,
                                                     update=False,
                                                     method=spec_method)
  for one_recom in spec_results:
    chebis = [val[0] for val in one_recom.candidates]
    forms = list(set([cn.REF_CHEBI2FORMULA[k] \
             for k in chebis if k in cn.REF_CHEBI2FORMULA.keys()]))
    pred_formulas[one_recom.id] = forms
# Predict reaction annotations. 
pred_reaction = self.reactions.predictAnnotation(inp_spec_dict=pred_formulas,
                                                 inp_reac_list=pred_ids,
                                                 update=update)
# pred_score = self.reactions.evaluatePredictedReactionAnnotation(pred_result=pred_reaction)
urls = {k:[cn.RHEA_DEFAULT_URL+val[0][5:] \
        for val in pred_reaction[cn.MATCH_SCORE][k]] \
        for k in pred_ids}
labels = {k:[cn.REF_RHEA2LABEL[val[0]] \
          for val in pred_reaction[cn.MATCH_SCORE][k]] \
          for k in pred_ids}
result = [cn.Recommendation(k,
                            # np.round(pred_score[k], cn.ROUND_DIGITS),
                            pred_reaction[cn.MATCH_SCORE][k],
                            urls[k],
                            labels[k]) \
          for k in pred_score.keys()]

In [9]:
recom.reactions.getRScores(spec_dict=recom.species.formula,
                           reacs=['ODC'])

{'ODC': [('RHEA:11584', 1.0),
  ('RHEA:14893', 1.0),
  ('RHEA:28827', 1.0),
  ('RHEA:71199', 1.0),
  ('RHEA:71527', 1.0),
  ('RHEA:72135', 1.0),
  ('RHEA:73471', 1.0),
  ('RHEA:22964', 0.5),
  ('RHEA:28891', 0.5),
  ('RHEA:34991', 0.5),
  ('RHEA:59048', 0.5),
  ('RHEA:70791', 0.5),
  ('RHEA:70799', 0.5),
  ('RHEA:70819', 0.5),
  ('RHEA:71575', 0.5),
  ('RHEA:72819', 0.5),
  ('RHEA:73475', 0.5),
  ('RHEA:73483', 0.5),
  ('RHEA:18645', 0.3333333333333333),
  ('RHEA:23652', 0.3333333333333333),
  ('RHEA:24368', 0.3333333333333333),
  ('RHEA:28791', 0.3333333333333333),
  ('RHEA:58284', 0.3333333333333333),
  ('RHEA:69140', 0.3333333333333333),
  ('RHEA:70787', 0.3333333333333333),
  ('RHEA:70815', 0.3333333333333333),
  ('RHEA:73479', 0.3333333333333333),
  ('RHEA:11236', 0.25),
  ('RHEA:12901', 0.25),
  ('RHEA:13201', 0.25),
  ('RHEA:13265', 0.25),
  ('RHEA:13877', 0.25),
  ('RHEA:13929', 0.25),
  ('RHEA:15349', 0.25),
  ('RHEA:15941', 0.25),
  ('RHEA:16017', 0.25),
  ('RHEA:16037', 0.25

In [68]:
def getRScores(spec_dict,
               reacs=None,
               ref_mat=REF_MAT):
  """
  Get a sorted list of
  Rhea-rScore tuples.
  [(RHEA:XXXXX, 1.0), etc.]
  
  Parameters
  ----------
  dict: inp_spec_dict
      Dictionoary, {species id: formula(str-list)}
  reacs: str-list
      IDs of reactions to predict annotatinos.
  ref_mat: pd.DataFrame
      Reference matrix
      
  Returns
  -------
  :dict
      {one_str: [(Rhea:XXXXX, 1.0), ...]}
  """
  # get dictionary of reaction ID: species component
  r2pred_spec_formulas = dict()
  for one_rid in reacs:
    r2pred_spec_formulas[one_rid] = {spec:spec_dict[spec] \
                                     for spec in recom.reactions.reaction_components[one_rid]}
  # prepare query df for prediction
  query_df = pd.DataFrame(0, 
                          index=ref_mat.columns,
                          columns=reacs)
  for one_rid in reacs:
    one_set_species = r2pred_spec_formulas[one_rid]
    # for each species element of the select reaction
    for spec_key in one_set_species.keys():
      one_spec = one_set_species[spec_key]
      # For each one_rid, set the values 1.0
      query_df.loc[[val for val in one_spec if val in query_df.index], one_rid] = 1
  multi_mat = ref_mat.dot(query_df)
  ref_rowsum = ref_mat.sum(1)
  # divided by the number of elements of the reference
  div_mat = multi_mat.divide(ref_rowsum, axis=0)
  
  rscores = dict()
  for reac in recom.getReactionIDs():
    reac_rscore = list(zip(div_mat.index, div_mat[reac]))
    reac_rscore.sort(key=operator.itemgetter(1), reverse=True)
    rscores[reac] = reac_rscore    
  return rscores

In [69]:
res = getRScores(spec_dict=recom.species.formula,
                      reacs=recom.getReactionIDs())

In [71]:
res['ODC']

[('RHEA:11584', 1.0),
 ('RHEA:14893', 1.0),
 ('RHEA:28827', 1.0),
 ('RHEA:71199', 1.0),
 ('RHEA:71527', 1.0),
 ('RHEA:72135', 1.0),
 ('RHEA:73471', 1.0),
 ('RHEA:22964', 0.5),
 ('RHEA:28891', 0.5),
 ('RHEA:34991', 0.5),
 ('RHEA:59048', 0.5),
 ('RHEA:70791', 0.5),
 ('RHEA:70799', 0.5),
 ('RHEA:70819', 0.5),
 ('RHEA:71575', 0.5),
 ('RHEA:72819', 0.5),
 ('RHEA:73475', 0.5),
 ('RHEA:73483', 0.5),
 ('RHEA:18645', 0.3333333333333333),
 ('RHEA:23652', 0.3333333333333333),
 ('RHEA:24368', 0.3333333333333333),
 ('RHEA:28791', 0.3333333333333333),
 ('RHEA:58284', 0.3333333333333333),
 ('RHEA:69140', 0.3333333333333333),
 ('RHEA:70787', 0.3333333333333333),
 ('RHEA:70815', 0.3333333333333333),
 ('RHEA:73479', 0.3333333333333333),
 ('RHEA:11236', 0.25),
 ('RHEA:12901', 0.25),
 ('RHEA:13201', 0.25),
 ('RHEA:13265', 0.25),
 ('RHEA:13877', 0.25),
 ('RHEA:13929', 0.25),
 ('RHEA:15349', 0.25),
 ('RHEA:15941', 0.25),
 ('RHEA:16017', 0.25),
 ('RHEA:16037', 0.25),
 ('RHEA:16441', 0.25),
 ('RHEA:20569', 0.

In [61]:
rscores = dict()
for reac in recom.getReactionIDs():
  
  reac_rscore = list(zip(reac_res.index, reac_res[reac]))
  reac_rscore.sort(key=operator.itemgetter(1), reverse=True)
  rscores[reac] = reac_rscore

In [63]:
recom.getReactionIDs()

['ODC',
 'SAMdc',
 'SSAT_for_S',
 'SSAT_for_D',
 'PAO_for_aD',
 'PAO_for_aS',
 'SpdS',
 'SpmS',
 'MAT',
 'VCoA',
 'VacCoA',
 'P_efflux',
 'aD_efflux']

In [67]:
rscores['P_efflux']

[('RHEA:72135', 1.0),
 ('RHEA:28827', 0.5),
 ('RHEA:28891', 0.5),
 ('RHEA:71575', 0.5),
 ('RHEA:18645', 0.3333333333333333),
 ('RHEA:69140', 0.3333333333333333),
 ('RHEA:11236', 0.25),
 ('RHEA:13929', 0.25),
 ('RHEA:22964', 0.25),
 ('RHEA:23412', 0.25),
 ('RHEA:23816', 0.25),
 ('RHEA:30707', 0.25),
 ('RHEA:33247', 0.25),
 ('RHEA:59048', 0.25),
 ('RHEA:12268', 0.2),
 ('RHEA:12436', 0.2),
 ('RHEA:12721', 0.2),
 ('RHEA:15037', 0.2),
 ('RHEA:18273', 0.2),
 ('RHEA:21936', 0.2),
 ('RHEA:25181', 0.2),
 ('RHEA:25808', 0.2),
 ('RHEA:25812', 0.2),
 ('RHEA:22284', 0.16666666666666666),
 ('RHEA:29995', 0.16666666666666666),
 ('RHEA:34111', 0.16666666666666666),
 ('RHEA:13633', 0.14285714285714285),
 ('RHEA:10000', 0.0),
 ('RHEA:10004', 0.0),
 ('RHEA:10008', 0.0),
 ('RHEA:10012', 0.0),
 ('RHEA:10016', 0.0),
 ('RHEA:10020', 0.0),
 ('RHEA:10024', 0.0),
 ('RHEA:10028', 0.0),
 ('RHEA:10032', 0.0),
 ('RHEA:10036', 0.0),
 ('RHEA:10040', 0.0),
 ('RHEA:10044', 0.0),
 ('RHEA:10048', 0.0),
 ('RHEA:10052', 0.

In [49]:
odc = reac_mult.loc[:,'ODC']
odc[odc==2]

RHEA:22964    2
RHEA:28827    2
RHEA:59048    2
Name: ODC, dtype: int64

In [52]:
ref_rowsum['RHEA:59048']

4

In [56]:
reac_res.loc['RHEA:59048', 'ODC']

0.5

In [37]:
for one_r in recom.getReactionIDs():
  print(one_r)
  print(np.max(reac_res[one_r]))
  print(np.min(reac_res[one_r]))
  print()

ODC
1.0
0.0

SAMdc
1.0
0.0

SSAT_for_S
1.0
0.0

SSAT_for_D
1.0
0.0

PAO_for_aD
1.0
0.0

PAO_for_aS
1.0
0.0

SpdS
1.0
0.0

SpmS
1.0
0.0

MAT
1.0
0.0

VCoA
1.0
0.0

VacCoA
1.0
0.0

P_efflux
1.0
0.0

aD_efflux
0.25
0.0



In [36]:
reac_res

,ODC,SAMdc,SSAT_for_S,SSAT_for_D,PAO_for_aD,PAO_for_aS,SpdS,SpmS,MAT,VCoA,VacCoA,P_efflux,aD_efflux
RHEA:10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RHEA:10004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RHEA:10008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RHEA:10012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RHEA:10016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
RHEA:74391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RHEA:74395,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RHEA:74399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RHEA:74403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
reac_res

,ODC,SAMdc,SSAT_for_S,SSAT_for_D,PAO_for_aD,PAO_for_aS,SpdS,SpmS,MAT,VCoA,VacCoA,P_efflux,aD_efflux
RHEA:10000,0,0,0,0,0,0,0,0,0,0,0,0,0
RHEA:10004,0,0,0,0,0,0,0,0,0,0,0,0,0
RHEA:10008,0,0,0,0,0,0,0,0,0,0,0,0,0
RHEA:10012,0,0,0,0,0,0,0,0,0,0,0,0,0
RHEA:10016,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
RHEA:74391,0,0,0,0,0,0,0,0,0,0,0,0,0
RHEA:74395,0,0,0,0,0,0,0,0,0,0,0,0,0
RHEA:74399,0,0,0,0,0,0,0,0,0,0,0,0,0
RHEA:74403,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
reac_res

,ODC,SAMdc,SSAT_for_S,SSAT_for_D,PAO_for_aD,PAO_for_aS,SpdS,SpmS,MAT,VCoA,VacCoA,P_efflux,aD_efflux
RHEA:10000,0,0,0,0,0,0,0,0,0,0,0,0,0
RHEA:10004,0,0,0,0,0,0,0,0,0,0,0,0,0
RHEA:10008,0,0,0,0,0,0,0,0,0,0,0,0,0
RHEA:10012,0,0,0,0,0,0,0,0,0,0,0,0,0
RHEA:10016,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
RHEA:74391,0,0,0,0,0,0,0,0,0,0,0,0,0
RHEA:74395,0,0,0,0,0,0,0,0,0,0,0,0,0
RHEA:74399,0,0,0,0,0,0,0,0,0,0,0,0,0
RHEA:74403,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
mat_div = reac_res['MAT']/ref_rowsum
np.min(mat_div)

0.0

In [33]:
reac_res.divide(ref_rowsum, axis=0)

,ODC,SAMdc,SSAT_for_S,SSAT_for_D,PAO_for_aD,PAO_for_aS,SpdS,SpmS,MAT,VCoA,VacCoA,P_efflux,aD_efflux
RHEA:10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RHEA:10004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RHEA:10008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RHEA:10012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RHEA:10016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
RHEA:74391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RHEA:74395,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RHEA:74399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RHEA:74403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
# below doesn't work, need to adjust 
ref_rowsum = ref_mat.sum(1)
reac_res/ref_rowsum

,MAT,ODC,PAO_for_aD,PAO_for_aS,P_efflux,RHEA:10000,RHEA:10004,RHEA:10008,RHEA:10012,RHEA:10016,...,RHEA:74403,RHEA:74407,SAMdc,SSAT_for_D,SSAT_for_S,SpdS,SpmS,VCoA,VacCoA,aD_efflux
RHEA:10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RHEA:10004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RHEA:10008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RHEA:10012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RHEA:10016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RHEA:74391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RHEA:74395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RHEA:74399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RHEA:74403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:

type(ref_rowsum)

pandas.core.series.Series

In [25]:
ref_rowsum.T

RHEA:10000    4
RHEA:10004    1
RHEA:10008    5
RHEA:10012    4
RHEA:10016    5
             ..
RHEA:74391    1
RHEA:74395    2
RHEA:74399    2
RHEA:74403    2
RHEA:74407    2
Length: 15451, dtype: int64

In [ ]:
ref_mat.

In [ ]:
    if inp_reac_list is not None:
      reaction_ids = inp_reac_list
    else:
      reaction_ids = list(self.reaction_components.keys())
    # get dictionary of reaction ID: species component
    r2pred_spec_formulas = dict()
    for one_rid in reaction_ids:
      r2pred_spec_formulas[one_rid] = {one_spec:inp_spec_dict[one_spec] \
                                       for one_spec in self.reaction_components[one_rid]}
    # prepare query df for prediction
    query_df = pd.DataFrame(0, 
                            index=inp_ref_mat.columns,
                            columns=reaction_ids)
    for one_rid in reaction_ids:
      one_set_species = r2pred_spec_formulas[one_rid]
      # for each species element of the select reaction
      for one_spec_key in one_set_species.keys():
        one_spec = one_set_species[one_spec_key]
        # For each one_rid, set the values 1.0
        query_df.loc[[val for val in one_spec if val in query_df.index], one_rid] = 1
        
        
    multi_mat = inp_ref_mat.dot(query_df)
    maxes = multi_mat.max()
    #
    # Collect candidates and calculate confidence score
    pred_cands = dict()
    pred_match_score = dict()
    for one_rid in maxes.index:
      one_multi = multi_mat.loc[:,one_rid]
      candidates = one_multi[one_multi==maxes[one_rid]].index
      # cand_data; (number of element matches, candidates)
      pred_cands[one_rid] = candidates
      # Now, match_scpre (calculated per each candidate) => replaced as a tuple
      match_score_per_cand = []
      for one_cand in candidates:
        if one_cand in cn.REF_RHEA2MASTER.keys():
          num_matches = maxes[one_rid]
          num_maxpos_matches = len(inp_ref_mat.loc[one_cand, :].to_numpy().nonzero()[0])
          match_score_per_cand.append((one_cand, np.round(num_matches/num_maxpos_matches, cn.ROUND_DIGITS)))
      match_score_per_cand.sort(key=operator.itemgetter(1), reverse=True)
      pred_match_score[one_rid] = match_score_per_cand
    if update:
      self.candidates = pred_match_score
      self.query_df = query_df

In [10]:
M_FDP_C = 'M_fdp_c'
M_ATP_C = 'M_atp_c'
recom2 = recommender.Recommender(libsbml_fpath=ecoli_fpath)

In [35]:
# two options: 
# 1. MSSC best with cutoff=0.0 (default)
# 2. MSSC above (cutoff=0.0?)

# species returns a list of ranked candidates with scores; 

In [44]:
spec_res = recom.getSpeciesRecommendation('oxygen', mssc='above', cutoff=0.01,
                                          get_df=True)
spec_res

,annotation,match score,label
oxygen (cred. 0.855),,,
1,CHEBI:15379,1.000,dioxygen
2,CHEBI:25805,1.000,oxygen atom
3,CHEBI:29194,0.866,monooxygen
4,CHEBI:33815,0.866,oxygen-18 atom
5,CHEBI:33818,0.866,oxygen-16 atom
...,...,...,...
145089,CHEBI:143987,0.037,"TG(22:5(7Z,10Z,13Z,16Z,19Z)/20:5(5Z,8Z,11Z,14Z..."
145090,CHEBI:149577,0.035,"PE 20:3(8Z,11Z,14Z)/20:5(5Z,8Z,11Z,14Z,17Z)"
145091,CHEBI:145857,0.034,"PE(22:2(13Z,16Z)/22:6(4Z,7Z,10Z,13Z,16Z,19Z))"


In [5]:
import editdistance
import operator
from AMAS.species_annotation import CHEBI_LOW_SYNONYMS, CHARCOUNT_DF, CHEBI_DF

In [6]:
pred = recom.species.getCScores(['hydrogen'])

In [7]:
recom.species.evaluatePredictedSpeciesAnnotation(
                                         pred=pred['hydrogen'],
                                         name_used='hydrogen',
                                         fitted_model=sa.SPECIES_RF)

0.9021236234028662

In [45]:
pred_strs=None
pred_ids=None
update=True
method='cdist'
get_df=False



In [22]:
recom.species.names

{'SAM': 'S-adenosyl-L-methionine',
 'A': 'S-adenosylmethioninamine',
 'P': 'Putrescine',
 'S': 'Spermine',
 'D': 'Spermidine',
 'aS': 'N1-Acetylspermine',
 'aD': 'N1-Acetylspermidine',
 'Met': 'Methionine',
 'ORN': 'L-Ornithine',
 'AcCoA': 'Acetyl-CoA',
 'CoA': 'CoA'}

In [87]:
def applyMSSC(pred,
              mssc,
              cutoff):
  """
  Apply MSSC to a predicted results. 
  
  Parameters
  ----------
  pred: list-tuple
      [(CHEBI:XXXXX, 1.0), etc.]
  mssc: string
  cutoff: float
  
  Returns
  -------
  filt: list-tuple
      [(CHEBI:XXXXX, 1.0), etc.]
  """
  filt_pred = [val for val in pred if val[1]>=cutoff]
  if not filt_pred:
    return []
  if mssc == 'top':
    max_val = np.max([val[1] for val in filt_pred])
    res_pred = [val for val in filt_pred if val[1]==max_val]
  elif mssc == 'above':
    res_pred = filt_pred
  return res_pred

In [95]:
pred_strs = ['S-adenosyl-L-methionine', 'S-adenosylmethioninamine']
# pred_strs = None
# pred_ids = ['SAM', 'A'] 
pred_ids = None
mssc = 'top'
cutoff = 0.0

scoring_methods = {'edist': recom.species.getEScores,
                   'cdist': recom.species.getCScores}
if pred_strs: 
  ids_dict = {k:k for k in pred_strs}
  inp_strs = pred_strs
elif pred_ids:
  ids_dict = {k:recom.species.getNameToUse(inp_id=k) \
              for k in pred_ids}
  inp_strs = [ids_dict[k] for k in ids_dict.keys()]
pred_res = scoring_methods[method](inp_strs)
# convert {name_used:[]} to {id:[]} and apply mssc
conv_res = {k:applyMSSC(pred_res[ids_dict[k]], mssc, cutoff) \
            for k in ids_dict.keys()}

result = []
for spec in conv_res.keys():
  pred_score = recom.species.evaluatePredictedSpeciesAnnotation(pred=conv_res[spec],
                                                                name_used=ids_dict[spec])
  urls = [cn.CHEBI_DEFAULT_URL + val[0][6:] for val in conv_res[spec]]
  labels = [cn.REF_CHEBI2LABEL[val[0]] for val in conv_res[spec]]
  one_recom = cn.Recommendation(spec,
                                np.round(pred_score, cn.ROUND_DIGITS),
                                [(val[0], np.round(val[1], cn.ROUND_DIGITS)) \
                                 for val in conv_res[spec]],
                                urls,
                                labels)
  result.append(one_recom)

In [96]:
result

[Recommendation(id='S-adenosyl-L-methionine', credibility=0.974, candidates=[('CHEBI:15414', 1.0), ('CHEBI:59789', 1.0)], urls=['https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI%3A15414', 'https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI%3A59789'], labels=['S-adenosyl-L-methionine', 'S-adenosyl-L-methionine zwitterion']),
 Recommendation(id='S-adenosylmethioninamine', credibility=0.958, candidates=[('CHEBI:15625', 1.0)], urls=['https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI%3A15625'], labels=['S-adenosylmethioninamine'])]

In [89]:
dummy_res = [('CHEBI:15414', 0.9),('CHEBI:59789', 0.5)]

In [93]:
applyMSSC(dummy_res, mssc='top', cutoff=0.8)

[('CHEBI:15414', 0.9)]

In [38]:
conv_res.keys()

dict_keys(['SAM', 'A'])

In [17]:
recom.species.getNameToUse(inp_id='SAM')

'S-adenosyl-L-methionine'

In [15]:
ids_dict

{None: None}

In [ ]:
    if method == 'edist':
      if pred_str:
        name_to_use = pred_str
        given_id = pred_str
      elif pred_id:
        name_to_use = self.species.getNameToUse(inp_id=pred_id)
        given_id = pred_id
      pred_res = self.species.predictAnnotationByEditDistance(name_to_use)  
    elif method == 'cdist':
      if pred_str: 
        given_id = pred_str
      elif pred_id:
        given_id = pred_id
      pred_res = self.species.predictAnnotationByCosineSimilarity(inp_strs=[given_id])[given_id]
    #
    pred_score = self.species.evaluatePredictedSpeciesAnnotation(pred_result=pred_res)
    urls = [cn.CHEBI_DEFAULT_URL + val[6:] for val in pred_res[cn.CHEBI]]
    labels = [cn.REF_CHEBI2LABEL[val] for val in pred_res[cn.CHEBI]]
    result = cn.Recommendation(given_id,
                               np.round(pred_score, cn.ROUND_DIGITS),
                               pred_res[cn.MATCH_SCORE],
                               urls,
                               labels)